<a href="https://colab.research.google.com/github/MarigoldJ/ygl2/blob/main/class/20210618_nlp_day7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Today's Topic

* RNN의 문제점
    * 기울기 소실
    * 번역에는 사용하기 힘들다?
        * 각 나라별 어순 정보 파악 힘들다.

# Seq2seq 구현하기

## LSTM Encoder

In [ ]:
import tensorflow as tf

In [ ]:
class Encoder(tf.keras.Model):
    '''
    seq2seq의 encoder
    '''
    def __init__(self, vocab_size, embedding_dim, encoder_units):
        super(Encoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(encoder_units)
        # return_sequences 매개변수를 기본값 False로 전달
    
    def call(self, x):      # __call__ 과 비슷한 듯
        '''
        x를 넣고 중간에 데이터의 shape을 디버깅
        '''
        print('입력할 때 shape :', x.shape)

        e = self.embedding(x)
        print("Embedding Layer를 거친 뒤 shape :", e.shape)

        output_v = self.lstm(e)
        print("LSTM Layer를 거친 뒤 shape :", output_v.shape)

        return output_v


In [ ]:
vocab_size = 30000
emb_size = 256
lstm_size = 512
batch_size = 1
sample_seq_len = 3

print('Vocab Size :', vocab_size)
print('Embedding Size :', emb_size)
print('LSTM Size :', lstm_size)
print('Batch Size :', batch_size)
print('Sample Sequence Length :', sample_seq_len)

Vocab Size : 30000
Embedding Size : 256
LSTM Size : 512
Batch Size : 1
Sample Sequence Length : 3


In [ ]:
encoder = Encoder(vocab_size, emb_size, lstm_size)
sample_encoder_input = tf.zeros((batch_size, sample_seq_len))

sample_encoder_output = encoder(sample_input)
# 인코더 LSTM의 최종 State (이후 컨벡스트 벡터로 사용될 예정)

입력할 때 shape : (1, 3)
Embedding Layer를 거친 뒤 shape : (1, 3, 256)
LSTM Layer를 거친 뒤 shape : (1, 512)


![](https://aiffelstaticprd.blob.core.windows.net/media/images/GN-4-L-6.max-800x600.jpg)

## LSTM Decoder

In [ ]:
class Decoder(tf.keras.Model):
    '''
    seq2seq의 decoder
    '''
    def __init__(self, vocab_size, embedding_dim, decoder_units):
        super(Decoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(decoder_units, return_sequences=True)
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.softmax = tf.keras.layers.Softmax(axis=-1)

    def call(self, x, context_v):
        '''
        디코더의 입력 x와 인코더의 컨벡스트 벡터(final state)를 인자로 받음
        중간의 데이터들의 shape을 디버깅
        '''
        print('입력할 때 shape :', x.shape)

        e = self.embedding(x)
        print("Embedding Layer를 거친 뒤 shape :", e.shape)

        # 컨벡스트 벡터의 브로드캐스팅, 확장
        context_vh = tf.repeat(tf.expand_dims(context_v, axis=1), repeats=x.shape[1], axis=1)
        eh = tf.concat([e, context_vh], axis=-1)
        print('Context Vector가 합쳐진 shape :', eh.shape)

        output_v = self.lstm(eh)
        print("LSTM Layer를 거친 뒤 shape :", output_v.shape)

        output = self.fc(output_v)
        print('Decoder 최종 output의 shape :', output.shape)

        return output

In [ ]:
vocab_size = 30000
emb_size = 256
lstm_size = 512
batch_size = 1
sample_seq_len = 3

print('Vocab Size :', vocab_size)
print('Embedding Size :', emb_size)
print('LSTM Size :', lstm_size)
print('Batch Size :', batch_size)
print('Sample Sequence Length :', sample_seq_len)

Vocab Size : 30000
Embedding Size : 256
LSTM Size : 512
Batch Size : 1
Sample Sequence Length : 3


In [ ]:
decoder = Decoder(vocab_size, emb_size, lstm_size)
sample_decoder_input = tf.zeros((batch_size, sample_seq_len))

sample_decoder_output = decoder(sample_decoder_input, sample_encoder_output)
# Decoder.call(x, context_v)를 호출


입력할 때 shape : (1, 3)
Embedding Layer를 거친 뒤 shape : (1, 3, 256)
Context Vector가 합쳐진 shape : (1, 3, 768)
LSTM Layer를 거친 뒤 shape : (1, 3, 512)
Decoder 최종 output의 shape : (1, 3, 30000)


# 어텐션 메커니즘 (Attention Mechanism)

* 기존의 RNN에 기반한 seq2seq 모델의 문제점
    1. 기억 소실, 기울기 소실
    2. 하나의 고정된 벡터에 모든 정보를 압축하려다 보니 정보 손실이 발생
* 어텐션 아이디어
    * 디코더에서 출력단어를 예측하는 매 시점마다, 인코더에서의 *전체* 입력 문장을 다시 한번 참고함
    * 전체 입력 문장을 모두 동일한 비율로 참고하는 것이 아니라, 해당 시점에 예측할 단어와 연관성 있는 단어 부분을 중점적으로 참고함

In [ ]:
dictionary = {'2017': 'Transformer', '2018': 'BERT'}

print(dictionary['2017'])
print(dictionary['2018'])

Transformer
BERT


![](https://wikidocs.net/images/page/22893/%EC%BF%BC%EB%A6%AC.PNG)
* Query : t 시점의 디코더 셀에서의 hidden state
* Key : 모든 시점의 인코더 셀에서의 hidden state
* Value : 모든 시점의 인코더 셀의 hidden state

## 닷 프로덕트 어텐션 (Dot-Product Attention)

* Luong이 제안한 어텐션 메커니즘
* score function이 dot product연산이므로, 이름이 dot-product attention이라 불림

### 이론 설명

![](https://wikidocs.net/images/page/22893/dotproductattention1_final.PNG)

그림 설명
* 디코더에서 sos, je를 넣어 je, suis를 예측한 뒤, 세번째 단어를 예측하려는 시점
* suis 다음 단어를 예측하기 위해서, 인코더의 hidden state들을 참조하여 다음 단어 etudiant를 예측해내는 모습

#### 1.Attention Score 구하기

![](https://wikidocs.net/images/page/22893/dotproductattention2_final.PNG)


그림 설명
* 디코더의 현재 시점 t에서의 state와, 인코더의 모든 시점에서의 state의 유사도를 점수화한다.(내적을 통해 유사도를 계산하는 듯)
* 수식은 아래와 같다.
$$ score(s_t, h_i) = s_t^T h_i $$
$$ e^t = [ score(s_t, h_1), score(s_t, h_2), ..., score(s_t, h_N) ] $$
    * $s_t$는 디코더의 현재 시점 t에서의 state (행벡터)
    * $h_i$는 인코더의 모든 시점 중 특정 시점 i에서의 state (행벡터)
    * $e^t$는 디코더의 현재 시점 t에서의 attention score 모음값


#### 2.Attention Distribution 구하기

* 소프트맥스(softmax) 함수를 통해 어텐션 분포를 구한다

![](https://wikidocs.net/images/page/22893/dotproductattention3_final.PNG)

그림 설명
* 1에서 계산한 각 시점별 Attention Score를 softmax 함수로 처리한다.
* 수식은 아래와 같다.
$$ a^t = softmax(e^t) $$
    * $a^t$는 어텐션 가중치 모음값인 attention distribution
    * $e^t$는 디코더의 현재 시점 t에서의 attention score 모음값

#### 3.Attention Value 구하기

* 각 인코더의 어텐션 가중치와 은닉 상태를 가중합해서 어텐션 값(attention value)을 구한다.

![](https://wikidocs.net/images/page/22893/dotproductattention4_final.PNG)

그림 설명
* 2에서 계산한 각 시점별 가중치인 Attention distribution과, 각 시점별 state를 곱하여 Attention value를 구한다.
    * 이 결과 값은 종합적인 정보를 담은 state라고 생각할 수 있다.
    * 인코더에서 디코더로 전달하는 state를 만든 것이다.
        * 기존 : 인코더의 마지막 state
        * 현재 하는 것 : 인코더의 state를 종합적으로 고려한 새로운 state
* 수식은 아래와 같다.
$$ a_t = \sum^{N}_{i=1}{a^t_i h_i} $$
    * $a^t$는 어텐션 가중치 모음값인 attention distribution
        * 해당 시점을 얼마나 반영할지 나타낸다고 생각하기
    * $h_i$는 인코더의 특정 시점 i에서의 state (행벡터)
    * $a_t$는 가중치가 반영된 state (행벡터)
        * $h_i$ 들을 대표하는 state라고 생각하면 좋을 듯

#### 4.Attention value, decoder state 연결하기

* 3에서 구한 어텐션 값과, 기존의 디코더의 현재 시점 t의 state를 연결한다.
* 새로운 벡터를 만드는 것이다.

![](https://wikidocs.net/images/page/22893/dotproductattention5_final_final.PNG)

그림 설명
* 3에서 계산한 인코더의 종합 State인 Attention Value와, 기존의 디코더의 현재 시점 t의 state를 연결하여 새로운 벡터를 만든다.
* 굳이 수식으로 쓰면 아래와 같다.
$$ v_t = [a_t;s_t] $$
    * $v_t$는 $a_t$와 $s_t$를 단순히 연결한 것. (행벡터)
    * $a_t$는 가중치가 반영된 인코더의 state (행벡터)
        * 인코더의 정보를 종합한 state로 보면 좋을 듯!
    * $s_t$는 디코더의 현재 시점 t에서의 state (행벡터)

#### 5.출력층 연산의 입력 벡터 만들기

* 논문 내용에 따르면, 4에서의 $v_t$가 출력층으로 가기전에 한가지 연산을 거침.
* 연산이라 함은 $tanh$ 함수를 거치는 연산을 의미함.

![](https://wikidocs.net/images/page/22893/st.PNG)

그림 설명
* 어텐션 메커니즘을 사용하기 전에는, 출력층의 입력으로 $s_t$(디코더에서 시점 t의 state) 가 사용됨.
* 어텐션 메커니즘을 사용하는 지금은, 출력층의 입력으로 $\tilde{s_t}$(attention이 반영된 디코더에서 시점 t의 state) 가 사용됨.
* $\tilde{s_t}$를 구하는 수식은 아래와 같다.
$$ \tilde{s_t}=tanh(W_c v_t + b_c) $$
    * $v_t$는 $a_t$와 $s_t$를 단순히 연결한 것. (행벡터)
    * $W_c$는 학습 가능한 가중치 행렬
    * $b_c$는 편향
    * $\tilde{s_t}$는 attention이 반영된 새로운 $s_t$(디코더의 현재 시점 t에서의 state) (행벡터)

#### 6.출력층으로부터 예측 벡터 얻기

* 5에서 구한 출력층의 입력벡터를 출력층에 넣고 연산하여, 예측 벡터($\tilde{y_t}$)를 얻는다.

* 수식은 아래와 같다. 
$$ \tilde{y_t}=softmax(W_y \tilde{s_t} + b_y) $$
    * $\tilde{s_t}$는 attention이 반영된 새로운 $s_t$(디코더의 현재 시점 t에서의 state) (행벡터)
    * $W_y$는 학습 가능한 가중치 행렬
    * $b_y$는 편향
    * $\tilde{y_t}$는 예측벡터 (이 값을 문자로 바꾸면 예측 문자가 된다)


## 바다나우 어텐션 (Bahdanau Attention, concat attention)

* Bahdanau가 제안한 어텐션 메커니즘
* score function이 concat 연산이라서 concat attention, 혹은 제안자 이름을 따서 Bahdanau attention이라 불림

* attention score를 얻는 식
$$ score_{alignment} = W * tanh(W_{decoder}*H_{decoder} + W_{encoder}*H_{encoder}) $$

### 코드 구현

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):    

    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.w_decoder = tf.keras.layers.Dense(units)
        self.w_encoder = tf.keras.layers.Dense(units)
        self.w_combine = tf.keras.layers.Dense(1)
    
    def call(self, h_encoder, h_decoder):
        '''
        h_encoder : encoder hidden state
        h_decoder : decoder hidden state
        '''
        wh_encoder = self.w_encoder(h_encoder)
        wh_decoder = self.w_decoder(tf.expand_dims(h_decoder, 1))

        print('[h_encoder] shape :', h_encoder.shape)
        print('[w_encoder x h_encoder] shape :', wh_encoder.shape)
        
        print('[h_decoder] shape :', h_decoder.shape)
        print('[w_decoder x h_decoder] shape :', wh_decoder.shape)

        # attention score
        at_score = self.w_combine(tf.nn.tanh(wh_decoder + wh_encoder))
        print('[score_alignment] shape :', at_score.shape)

        # attention distribution(weight)
        at_weight = tf.nn.softmax(at_score, axis=1)
        print('\n최종 attention weight :', at_weight.numpy())

        # attention value?(convext vector 얻기)
        context_v = at_weight * wh_decoder
        context_v = tf.reduce_sum(context_v, axis=1)

        return context_v, at_weight


In [ ]:
w_size = 100

print(f'Hidden State를 {w_size}차원으로 Mapping\n')

attention = BahdanauAttention(w_size)

encode_state = tf.random.uniform((1, 10, 512))
decode_state = tf.random.uniform((1, 512))

_ = attention(encode_state, decode_state)


Hidden State를 100차원으로 Mapping

[h_encoder] shape : (1, 10, 512)
[w_encoder x h_encoder] shape : (1, 10, 100)
[h_decoder] shape : (1, 512)
[w_decoder x h_decoder] shape : (1, 1, 100)
[score_alignment] shape : (1, 10, 1)

최종 attention weight : [[[0.09870458]
  [0.13317658]
  [0.10386166]
  [0.1029733 ]
  [0.09819912]
  [0.09062405]
  [0.07663803]
  [0.09599996]
  [0.07814394]
  [0.12167887]]]


## 루옹 어텐션 (Luong Attention)

* Luong이 제안한 어텐션 메커니즘
* score function 이름이 general임.

* dot-product attention도 Luong이 제안했지만, 지금 확인 중인 attention도 Luong이 제안함. 
    * 둘의 차이는 score function에 가중치 $W_a$ 존재 여부임.

* attention score를 얻는 식
$$ score(H_{target}, H_{source}) = H_{target}^T*W_{combine}*H_{source} $$

### 코드 구현

In [ ]:
class LuongAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(LuongAttention, self).__init__()
        self.W_combine = tf.keras.layers.Dense(units)

    def call(self, h_encoder, h_decoder):
        '''
        h_encoder : encoder hidden state (식에서 H_source)
        h_decoder : decoder hidden state (식에서 H_target)
        '''
        # attention score
        # 우선 W_combine * H_source를 수행
        wh_combine = self.W_combine(h_encoder)  # W_combine * H_source

        print('[h_source] shape :', h_encoder.shape)
        print('[w_combine x h_source] shape :', wh_combine.shape)

        # H_target.T와 W_combine * H_source의 행렬곱 수행 (=score)
        h_decoder_rev = tf.expand_dims(h_decoder, 1)                                    # 차원 수가 달라서, 차원 확장
        at_score_rev = tf.matmul(wh_combine, tf.transpose(h_decoder_rev, [0, 2, 1]))    # 결과값이 10개 score 값이 나오도록 행렬곱
        
        print('[score_alignment] shape :', at_score_rev.shape)  # (1, 10, 1)

        # attention distribution(weight)
        at_weight_rev = tf.nn.softmax(at_score_rev, axis=1)

        print('\n최종 attention weight :', at_weight_rev.numpy())

        # 늘렸던 차원수 다시 맞춰주기
        at_weight = tf.squeeze(at_weight_rev, axis=-1)          # (1, 10)

        # attention value?(convext vector 얻기)
        context_v = tf.matmul(at_weight, h_encoder)     # (1, 10) * (1, 10, 512) = (1, 512)

        return context_v, at_weight


In [ ]:
emb_dim = 512

attention = LuongAttention(emb_dim)

enc_state = tf.random.uniform((1, 10, emb_dim))
dec_state = tf.random.uniform((1, emb_dim))

_ = attention(enc_state, dec_state)

[h_source] shape : (1, 10, 512)
[w_combine x h_source] shape : (1, 10, 512)
[score_alignment] shape : (1, 10, 1)

최종 attention weight : [[[9.6180429e-06]
  [8.6220843e-04]
  [3.5097243e-03]
  [9.5622498e-01]
  [1.9150067e-03]
  [3.5099234e-02]
  [1.4689063e-03]
  [2.9138316e-06]
  [1.7087453e-04]
  [7.3649467e-04]]]


# 양방향 LSTM과 어텐션 메커니즘 (IMDB 리뷰데이터)

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [ ]:
print('리뷰의 최대 길이 :', max(len(line) for line in x_train))
print('리뷰의 평균 길이 :', sum(map(len, x_train)) / len(x_train))

리뷰의 최대 길이 : 2494
리뷰의 평균 길이 : 238.71364


In [ ]:
max_len = 500
x_train_padding = pad_sequences(x_train, maxlen=max_len)
x_test_padding = pad_sequences(x_test, maxlen=max_len)

## 바다나우 어텐션 (Bahdanau Attention)

* attention score를 얻는 식
$$ score(query, key) = W^T tanh(W_1 key + W_2 query) $$

In [ ]:
import tensorflow as tf

In [ ]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)
    

    def call(self, values, query):
        '''
        shape 정리
            query: (batch_size, hidden_size)

            hidden with time axis: (batch_size, 1, hidden_size)
            at_score: (batch_size, max_length, 1)
            at_weight: (batch_size, max_length, 1)
            context_v: (batch_size, hidden_size)
        '''

        hidden_with_time_axis = tf.expand_dims(query, 1)

        at_score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        at_weight = tf.nn.softmax(at_score, axis=1)

        context_v = tf.reduce_sum(at_weight * values, axis=1)

        return context_v, at_weight
    

## 양방향 LSTM + 어텐션 메커니즘

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

In [ ]:
sequence_input = Input(shape=(max_len,), dtype='int32')

embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero=True)(sequence_input)
lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True))(embedded_sequences)
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)


In [ ]:
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)


(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


In [ ]:
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

In [ ]:
attention = BahdanauAttention(64)
context_vector, attention_weights = attention(lstm, state_h)

In [ ]:
dense1 = Dense(20, activation='relu')(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation='sigmoid')(dropout)
model = Model(inputs=sequence_input, outputs=output)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train_padding, y_train, epochs=5, batch_size=256,
                    validation_data=(x_test_padding, y_test), verbose=1)

Epoch 1/5
98/98 [==============================] - 644s 6s/step - loss: 0.4683 - accuracy: 0.7651 - val_loss: 0.2885 - val_accuracy: 0.8800
Epoch 2/5
98/98 [==============================] - 619s 6s/step - loss: 0.2421 - accuracy: 0.9134 - val_loss: 0.2951 - val_accuracy: 0.8787
Epoch 3/5
98/98 [==============================] - 621s 6s/step - loss: 0.1858 - accuracy: 0.9365 - val_loss: 0.3058 - val_accuracy: 0.8771
Epoch 4/5
98/98 [==============================] - 613s 6s/step - loss: 0.1570 - accuracy: 0.9471 - val_loss: 0.3428 - val_accuracy: 0.8664
Epoch 5/5
98/98 [==============================] - 605s 6s/step - loss: 0.1249 - accuracy: 0.9599 - val_loss: 0.3936 - val_accuracy: 0.8733


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test_padding, y_test)[1]))

782/782 [==============================] - 409s 523ms/step - loss: 0.3936 - accuracy: 0.8733

 테스트 정확도: 0.8733


# seq2seq with attention 스페인-영어 번역기

## 데이터 준비하기

In [ ]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

In [ ]:
path_to_zip = tf.keras.utils.get_file('spa-eng.zip', extract=True,
                                      origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip')

In [ ]:
path_to_file = os.path.dirname(path_to_zip) + '/spa-eng/spa.txt'

with open(path_to_file, 'r') as f:
    raw = f.read().splitlines()

print('Data Size :', len(raw))
print('Example :')

for sen in raw[0:100][::20]:
    print('>>', sen)

Data Size : 118964
Example :
>> Go.	Ve.
>> Wait.	Esperen.
>> Hug me.	Abrázame.
>> No way!	¡Ni cagando!
>> Call me.	Llamame.


## 데이터 전처리 : 정제하기

In [ ]:
def preprocess_sentence(raw_sentence, s_token=False, e_token=False):
    sentence = raw_sentence.lower().strip()

    sentence = re.sub(r'([?.!,])', r' \1 ', sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r'[^a-zA-Z?.!,]+', ' ', sentence)
    sentence = sentence.strip()

    if s_token:
        sentence = '<start> ' + sentence
    if e_token:
        sentence = sentence + ' <end>'

    return sentence


In [ ]:
encode_corpus = []
decode_corpus = []

num_examples = 30000

for pair in raw[:num_examples]:
    eng, spa = pair.split('\t')

    encode_corpus.append(preprocess_sentence(eng))
    decode_corpus.append(preprocess_sentence(spa, s_token=True, e_token=True))

print('English :', encode_corpus[100])
print('Spanish :', decode_corpus[100])


English : go away !
Spanish : <start> salga de aqu ! <end>


## 데이터 전처리 : 토큰화

In [ ]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, tokenizer

In [ ]:
# 토큰화 하기
encode_tensor, encode_tokenizer = tokenize(encode_corpus)
decode_tensor, decode_tokenizer = tokenize(decode_corpus)

# 훈련데이터와 검증데이터로 분리하기
encode_train, encode_val, decode_train, decode_val = \
    train_test_split(encode_tensor, decode_tensor, test_size=0.2)

print('English Vocab size :', len(encode_tokenizer.index_word))
print('Spanish Vocab size :', len(decode_tokenizer.index_word))
print()
print('train length :', len(encode_train))
print('valid length :', len(encode_val))

English Vocab size : 4931
Spanish Vocab size : 8893

train length : 24000
valid length : 6000


In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):

    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.w_decode = tf.keras.layers.Dense(units)
        self.w_encode = tf.keras.layers.Dense(units)
        self.w_combine = tf.keras.layers.Dense(1)
    
    def call(self, h_encode, h_decode):
        '''
        h_encode : (batch, length, units)
        h_decode : (batch, units)
        '''
        wh_encode = self.w_encode(h_encode)
        wh_decode = self.w_decode(tf.expand_dims(h_decode, 1))

        # attention score
        at_score = self.w_combine(tf.nn.tanh(wh_decode + wh_encode))

        # attention distribution(weight)
        at_weight = tf.nn.softmax(at_score, axis=1)

        # context vector
        context_v = at_weight * wh_encode
        context_v = tf.reduce_sum(context_v, axis=1)

        return context_v, at_weight



![](https://aiffelstaticprd.blob.core.windows.net/media/images/GN-4-P-2.max-800x600.jpg)

In [ ]:
class Encoder(tf.keras.Model):
    
    def __init__(self, vocab_size, embedding_dim, encode_units):
        super(Encoder, self).__init__()
        self.encode_units = encode_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(encode_units, return_sequences=True)
        # return_sequences=True : 3차원 돌려줌

    def call(self, encode_input):

        embedded_input = self.embedding(encode_input)
        encode_output = self.gru(embedded_input)

        # shape 디버깅
        # print('input shape :', encode_input.shape)
        # print('after embedding ->', embedded_input.shape)
        # print('after lstm ->', encode_output.shape)

        return encode_output


In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, decode_units):
        super(Decoder, self).__init__()
        self.decode_units = decode_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(decode_units, return_sequences=True, return_state=True)
        self.attention = BahdanauAttention(self.decode_units)
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, decode_input, h_decode, encode_output):
        
        context_v, at_weight = self.attention(encode_output, h_decode)

        embedded_input = self.embedding(decode_input)

        v = tf.concat([tf.expand_dims(context_v, 1), embedded_input], axis=-1)

        decode_output, h_decode = self.gru(v)

        text_output = self.fc(tf.reshape(decode_output, (-1, decode_output.shape[2])))

        return text_output, h_decode, at_weight
        

In [ ]:
BATCH_SIZE = 64
src_vocab_size = len(encode_tokenizer.index_word) + 1
tgt_vocab_size = len(decode_tokenizer.index_word) + 1

units = 1024
embedding_dim = 512

encoder = Encoder(src_vocab_size, embedding_dim, units)
decoder = Decoder(tgt_vocab_size, embedding_dim, units)

# sample input
sequence_len = 30

sample_encode = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_output = encoder(sample_encode)
print()
print('Encoder Output :', sample_output.shape)

sample_state = tf.random.uniform((BATCH_SIZE, units))
sample_logits, h_decode, at_weight = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_state, sample_output)
print('Decoder Output :', sample_logits.shape)
print('Decoder Hidden State :', h_decode.shape)
print('Ateention weight :', at_weight.shape)


Encoder Output : (64, 30, 1024)
Decoder Output : (64, 8894)
Decoder Hidden State : (64, 1024)
Ateention weight : (64, 30, 1)


## 훈련하기 1. Optimizer & loss

* Categorical Crossentropy()
    * [0.1, 0.2, 0.7] --> one-hot encoding [0, 0, 1]
* SparseCategoricalCrossentropy()
    * [0.1, 0.2, 0.7] --> 정수 인덱스 2
    * `from_logits=True` --> 모델의 출력값을 그대로 전달

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask

    return tf.reduce_mean(loss)


## 훈련하기 2. train_step

* train_step 학습 과정
    1. Encoder에 소스 문장을 전달해 컨텍스트 벡터인 enc_out을 생성
    2. Decoder에 입력으로 전달할 `<start>`토큰 문장 생성
    3. t=0일대, Decoder의 Hidden State는 Encoder의 Final State로 정의. h_dec = enc_out[:, -1]
    4. 문장과 enc_out, Hidden State를 기반으로 다음단어 (t=1) 예측. pred
    5. 예측된 단어와 정답 간의 Loss를 구한 후, t=1의 정답 단어를 다음 입력으로 사용 (예측 단어 X)
    6. 반복하기!

In [ ]:
@tf.function
def train_step(src, tgt, encoder, decoder, optimzer, dec_tok):
    bsz = src.shape[0]
    loss = 0

    with tf.GradientTape() as tape: # 학습 과정 기록
        enc_out = encoder(src)
        h_dec = enc_out[:, -1]

        dec_src = tf.expand_dims([dec_tok.word_index['<start>']] * bsz, 1)

        for t in range(1, tgt.shape[1]):
            pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)

            loss += loss_function(tgt[:, t], pred)
            dec_src = tf.expand_dims(tgt[:, t], 1)
    
    batch_loss = (loss / int(tgt.shape[1]))
    
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [ ]:
# 돌리면 렉 오지게 걸리는 코드

from tqdm import tqdm
import random

epochs = 10

for epoch in range(epochs):
    total_loss = 0

    idx_list = list(range(0, encode_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss = train_step(encode_train[idx:idx+BATCH_SIZE],
                                decode_train[idx:idx+BATCH_SIZE],
                                encoder,
                                decoder,
                                optimizer,
                                decode_tokenizer)
        total_loss += batch_loss

        t.set_description_str(f'Epoch {epoch+1:2d}')
        t.set_postfix_str(f'Loss : {(total_loss.numpy() / (batch+1)):.4f}')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.


Epoch  2:   0%|          | 1/375 [00:00<01:11,  5.26it/s, Loss : 1.0662]



Epoch  2:   0%|          | 1/375 [00:00<01:11,  5.26it/s, Loss : 1.0662]



Epoch  2:   0%|          | 1/375 [00:00<01:11,  5.26it/s, Loss : 1.0320]



Epoch  2:   1%|          | 2/375 [00:00<01:11,  5.18it/s, Loss : 1.0320]



Epoch  2:   1%|          | 2/375 [00:00<01:11,  5.18it/s, Loss : 1.0320]



Epoch  2:   1%|          | 2/375 [00:00<01:11,  5.18it/s, Loss : 1.0072]



Epoch  2:   1%|          | 3/375 [00:00<01:14,  5.01it/s, Loss : 1.0072]



Epoch  2:   1%|          | 3/375 [00:00<01:14,  5.01it/s, Loss : 1.0072]



Epoch  2:   1%|          | 3/375 [00:00<01:14,  5.01it/s, Loss : 1.0159]



Epoch  2:   1%|          | 4/375 [00:00<01:11,  5.17it/s, Loss : 1.0159]



Epoch  2:   1%|          | 4/375 [00:00<01:11,  5.17it/s, Loss : 1.0159]



Epoch  2:   1%|          | 4/375 [00:00<01:11,  5.17it/s, Loss : 1.0259]



Epoch  2:   1%|▏         | 5/375 [00:00<01:12,  5.

## Evaluation step

In [ ]:
@tf.function
def eval_step(src, tgt, encoder, decoder, dec_tok):
    bsz = src.shape[0]
    loss = 0

    enc_out = encoder(src)

    h_dec = enc_out[:, -1]
    
    dec_src = tf.expand_dims([dec_tok.word_index['']] * bsz, 1)

    for t in range(1, tgt.shape[1]):
        pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)

        loss += loss_function(tgt[:, t], pred)
        dec_src = tf.expand_dims(tgt[:, t], 1)
        
    batch_loss = (loss / int(tgt.shape[1]))
    
    return batch_loss


# Training Process

from tqdm import tqdm

EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss = train_step(enc_train[idx:idx+BATCH_SIZE],
                                dec_train[idx:idx+BATCH_SIZE],
                                encoder,
                                decoder,
                                optimizer,
                                dec_tokenizer)
    
        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))
    
    test_loss = 0
    
    idx_list = list(range(0, enc_val.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (test_batch, idx) in enumerate(t):
        test_batch_loss = eval_step(enc_val[idx:idx+BATCH_SIZE],
                                    dec_val[idx:idx+BATCH_SIZE],
                                    encoder,
                                    decoder,
                                    dec_tokenizer)
    
        test_loss += test_batch_loss

        t.set_description_str('Test Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Test Loss %.4f' % (test_loss.numpy() / (test_batch + 1)))

## Attention Map 시각화

In [ ]:
def evaluate(sentence, encoder, decoder):
    attention = np.zeros((dec_train.shape[-1], enc_train.shape[-1]))
    
    sentence = preprocess_sentence(sentence)
    inputs = enc_tokenizer.texts_to_sequences([sentence.split()])
    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')

    result = ''

    enc_out = encoder(inputs)

    dec_hidden = enc_out[:, -1]
    dec_input = tf.expand_dims([dec_tokenizer.word_index['<start>']], 0)

    for t in range(dec_train.shape[-1]):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention[t] = attention_weights.numpy()

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0]).numpy()

        result += dec_tokenizer.index_word[predicted_id] + ' '

        if dec_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence, attention

        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention


def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def translate(sentence, encoder, decoder):
    result, sentence, attention = evaluate(sentence, encoder, decoder)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    
    attention = attention[:len(result.split()), :len(sentence.split())]
    plot_attention(attention, sentence.split(), result.split(' '))


In [ ]:
translate("Can I have some coffee?", encoder, decoder)

# 숙제

## 주말동안 공부해야할 숙제
한-영 번역기 만들기
1. 데이터 다운로드
- 데이터 : https://github.com/jungyeul/korean-parallel-corpora/tree/master/korean-english-news-v1
- korean-english-park.train.tar.gz
2. 데이터 정제
- set 데이터형이 중복이 허용하지 않다는 것을 활용해 중복된 데이터를 제거
  - 데이터 병렬 쌍이 흐트러지지 않게 주의!
  - cleaned_corpus에 저장
- 앞서 정의한 preprocessing()함수는 한글에 대해 동작하지 않아요.
  - 한글에 적용할 수 있는 정규식을 추가해여 함수를 재정의 하세요.
- 타겟 언어인 영문엔 <start>토큰과 <end>토큰을 추가하고 split()함수로 토큰화 합니다. 한글 토큰화는 konlpy의 mecab클래스를 사용합니다.
  - cleaned_corpus로부터 토큰의길이가 40이하인 데이터를 선별하여 eng_corpus와 kor_corpus를 각각 구축하기

3. 토큰화
- tokenize()함수를 사용해 데이터를 텐서로 변환하고 각각의 tokenizer를 얻으세요!
  - 단어수는 실험을 통해 적당한 값을 맞춰줍시다(최소 10000이상!)
4. 훈련하기

```
sudo apt -qq -y install fonts-nanum
```

```
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()
```